In [ ]:
from db import connection, engine
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [ ]:
pd.DataFrame(engine.table_names())

In [ ]:
zuschlag = pd.read_sql("SELECT * FROM zuschlag;",
    connection)
zuschlag

In [ ]:
grouped_zuschlag = zuschlag.groupby(['sprache']).size()
total = sum(grouped_zuschlag)
plt.title("Titel")
grouped_zuschlag.plot.pie(figsize=(6,6), autopct=lambda p: '{:.0f}'.format(p * total / 100))

In [ ]:
selects = "anbieter.*, zuschlag.preis, zuschlag.gatt_wto, zuschlag.meldungsnummer as z_meldungsnummer, zuschlag.anzahl_angebote, auftraggeber.institution, auftraggeber.beschaffungsstelle_plz, projekt.projekt_titel, cpv_dokument.cpv_nummer, cpv.cpv_deutsch"

query = """select {} from (((((beruecksichtigteanbieter_zuschlag
    INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
    INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
    INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
    INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
    INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id
    INNER JOIN cpv_dokument ON zuschlag.meldungsnummer = cpv_dokument.meldungsnummer)
    INNER JOIN cpv ON cpv_dokument.cpv_nummer = cpv.cpv_nummer
    ;
""".format(selects)

df = pd.read_sql(query, connection)

In [ ]:
selects = "anbieter.*, zuschlag.*, projekt.*, auftraggeber.*, ausschreibung.*, cpv_dokument.*"

query = """select {} from ((((((beruecksichtigteanbieter_zuschlag
    INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
    INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
    INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
    INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
    INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
    INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = zuschlag.meldungsnummer)
    ;
""".format(selects)

full_df = pd.read_sql(query, connection)
full_df

In [ ]:
query = """
SELECT 
    table_name, 
    table_rows 
FROM 
    INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'simap' AND table_name in ('zuschlag', 'ausschreibung', 'anbieter', 'auftraggeber', 'cpv_dokument', 'projekt');
"""
desc = pd.read_sql(query, connection)
desc

In [ ]:
desc.plot()

In [ ]:
test_df = full_df.copy()

In [ ]:
def tonumeric(val):
    try:
        return int(val)
    except:
        return 0
        
test_df[['anbieter_plz']] = test_df[['anbieter_plz']].applymap(tonumeric)

In [ ]:
pd.DataFrame(test_df.anbieter_plz.value_counts())

In [ ]:
plz_df = test_df[(test_df['anbieter_plz']<=9999) & (test_df['anbieter_plz']>=1000)]

In [ ]:
def plotCluster(df):
    
    filtered_df = df[['cpv_nummer','anbieter_plz']]
    scaler = StandardScaler().fit(filtered_df)
    cpv_filtered_z = scaler.transform(filtered_df)

    kmeans = KMeans().fit(cpv_filtered_z)
    y_means = kmeans.predict(cpv_filtered_z)

    df.plot.scatter(x='anbieter_plz',y='cpv_nummer', c=y_means, cmap='viridis' )
    df['label']=y_means




In [ ]:
plotCluster(plz_df)

In [ ]:
plz_df

In [ ]:
selects = (
    "anbieter.anbieter_id, "
    "cpv_dokument.cpv_nummer AS anbieter_cpv"
)

query = """
        SELECT {} from ((((((beruecksichtigteanbieter_zuschlag
        INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
        INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
        INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
        INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
        INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
        INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = zuschlag.meldungsnummer)
        WHERE anbieter.anbieter_id = 1;
        ;
""".format(selects)

df = pd.read_sql(query, connection)
df

In [ ]:
selects = (
    "anbieter.anbieter_id, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv"
)

query = """
        SELECT DISTINCT {} from ((((((beruecksichtigteanbieter_zuschlag
        INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
        INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
        INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
        INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
        INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
        INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = ausschreibung.meldungsnummer)
        WHERE anbieter.anbieter_id = 1;
        ;
""".format(selects)

df = pd.read_sql(query, connection)
df

In [ ]:
select1 = (
    "anbieter.anbieter_id, "
    "cpv_dokument.cpv_nummer as anbieter_cpv"
)
select2 = (
    "anbieter.anbieter_id, "
    "cpv_dokument.cpv_nummer as ausschreibung_cpv"
)

query = """
    SELECT * FROM (SELECT DISTINCT {} from ((((((beruecksichtigteanbieter_zuschlag
        INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
        INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
        INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
        INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
        INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
        INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = zuschlag.meldungsnummer)
        WHERE anbieter.anbieter_id = 1
    ) zuschlag
    JOIN (SELECT DISTINCT {} from ((((((beruecksichtigteanbieter_zuschlag
        INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
        INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
        INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
        INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
        INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
        INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = ausschreibung.meldungsnummer)
        INNER JOIN cpv on cpv.cpv_nummer = cpv_dokument.cpv_nummer
        WHERE anbieter.anbieter_id = 1
    ) ausschreibung ON ausschreibung.anbieter_id = zuschlag.anbieter_id
    ;
""".format(select1, select2)

df = pd.read_sql(query, connection)
df

In [ ]:
df['match'] = True
df

In [ ]:
full_df.columns

In [ ]:
selects = "anbieter.institution as inst, zuschlag.*, projekt.*, auftraggeber.*, ausschreibung.*, cpv_dokument.*"

query = """select {} from ((((((beruecksichtigteanbieter_zuschlag
    INNER JOIN zuschlag ON zuschlag.meldungsnummer = beruecksichtigteanbieter_zuschlag.meldungsnummer)
    INNER JOIN anbieter ON beruecksichtigteanbieter_zuschlag.anbieter_id = anbieter.anbieter_id)
    INNER JOIN projekt ON zuschlag.projekt_id = projekt.projekt_id)
    INNER JOIN auftraggeber ON projekt.auftraggeber_id = auftraggeber.auftraggeber_id)
    INNER JOIN ausschreibung ON projekt.projekt_id = ausschreibung.projekt_id)
    INNER JOIN cpv_dokument ON cpv_dokument.meldungsnummer = zuschlag.meldungsnummer)
    ;
""".format(selects)

full_df = pd.read_sql(query, connection)
full_df

In [ ]:
full_df[full_df['inst']=="Adecco AG"]['cpv_nummer'].value_counts()